In [1]:
import movingpandas as mpd
import geopandas as gp
import pandas as pd
from datetime import timedelta
from shapely import LineString, Point
import skmob
from skmob.preprocessing import detection
import numpy as np
from tqdm import tqdm

In [ ]:
# read geolife pickle file
print("Reading geolife pickle file...")
geolife_raw_gdf = gp.GeoDataFrame(pd.read_pickle('../data/geolife/geolife_raw.pkl')).to_crs(epsg=4326)
print("Done.")


In [28]:
def splitTrajectories(geolife_raw_gdf, max_diameter=100, min_duration_minutes=15, min_length=200, to_csv=False):
    """This function splits the trajectories into smaller segments using the movingpandas library. The split is done based on the stop points.


    Args:
        geolife_raw_gdf (_type_): Geolife raw data as geodataframe.
        max_diameter (int, optional): See movingpandas documentation. Defaults to 100.
        min_duration_minutes (int, optional): See movingpandas documentation. Defaults to 15.
        min_length (int, optional): See movingpandas documentation. Defaults to 200.

    Returns:
        _type_: Geodataframe with split trajectories and user id. 
    """
    traj_collection = mpd.TrajectoryCollection(geolife_raw_gdf, 'user', t='time')
    print("Trajectory collection created.")
    split_traj = []
    if to_csv:
        
        for index, traj in enumerate(traj_collection.trajectories):
            # split trajectory
            split = mpd.StopSplitter(traj).split(max_diameter=max_diameter, min_duration=timedelta(minutes=min_duration_minutes), min_length=min_length)

            split_traj = split.to_traj_gdf()

            # add user id to each split trajectory
            split_traj['user_id'] = split_traj['traj_id'].str.split('_').str[0]

            # save to csv
            split_traj.to_csv(f'data/geolife/split_trajectories/split_geolife_{split_traj["user_id"].unique()[0]}.csv', index=False)
            print(f"Next trajectory split and saved to csv.: {index}/{len(traj_collection.trajectories)}")
        return print("All split trajectories are saved to csv in the data folder.")
    else:
        for traj in tqdm(traj_collection.trajectories):
            # split trajectory
            split = mpd.StopSplitter(traj).split(max_diameter=max_diameter, min_duration=timedelta(minutes=min_duration_minutes), min_length=min_length)

            # add split trajectories to list after converting to geodataframe
            split_traj.append(split.to_traj_gdf())

            print(f"Next trajectory split and appended to list: {index}/{len(traj_collection.trajectories)}")

        # concat all split trajectories
        split_traj = pd.concat(split_traj)

        # add user id to each split trajectory
        split_traj['user_id'] = split_traj['traj_id'].str.split('_').str[0]
        print("All split trajectories and concatenated.")
        print("Done.")
        return split_traj

In [29]:
print("Splitting trajectories...")
splitTrajectories(geolife_raw_gdf, to_csv=True)
print("Done.")

Splitting trajectories...
Trajectory collection created.


1it [10:35, 635.88s/it]

Next trajectory split and saved to csv.: 0/182


2it [10:37, 262.86s/it]

Next trajectory split and saved to csv.: 1/182


...

161it [12:59:16, 144.91s/it]

Next trajectory split and saved to csv.: 160/182


162it [12:59:16, 288.62s/it]

Next trajectory split and saved to csv.: 161/182
All split trajectories are saved to csv in the data folder.


Done.
